In [24]:
def adjust_mmse(value):
    integer_part = int(value)  # Phần nguyên
    decimal_part = value - integer_part  # Phần thập phân
    
    if decimal_part > 0.6:
        integer_part += 1  # Cộng thêm 1 vào phần nguyên nếu phần thập phân > 0.6
    
    return integer_part

In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.inspection import permutation_importance
import xgboost as xgb
import matplotlib.pyplot as plt
from sksurv.util import Surv
import xgboost as xgb
df=pd.read_csv(r'D:\Python\python_work\AI_machine learning\alzheimers_disease_data.csv')
df['MMSE'] = df['MMSE'].apply(adjust_mmse)
df['PhysicalActivity']=df['PhysicalActivity'].apply(adjust_mmse)
print(df.columns)
X=df[['MMSE','FunctionalAssessment','MemoryComplaints','BehavioralProblems','ADL']]
y=df['Diagnosis']

Index(['PatientID', 'Age', 'Gender', 'Ethnicity', 'EducationLevel', 'BMI',
       'Smoking', 'AlcoholConsumption', 'PhysicalActivity', 'DietQuality',
       'SleepQuality', 'FamilyHistoryAlzheimers', 'CardiovascularDisease',
       'Diabetes', 'Depression', 'HeadInjury', 'Hypertension', 'SystolicBP',
       'DiastolicBP', 'CholesterolTotal', 'CholesterolLDL', 'CholesterolHDL',
       'CholesterolTriglycerides', 'MMSE', 'FunctionalAssessment',
       'MemoryComplaints', 'BehavioralProblems', 'ADL', 'Confusion',
       'Disorientation', 'PersonalityChanges', 'DifficultyCompletingTasks',
       'Forgetfulness', 'Diagnosis', 'DoctorInCharge'],
      dtype='object')


In [63]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
cat_columns=['PatientID','Ethnicity','DoctorInCharge','Diagnosis']
new_df=df.drop(cat_columns,axis=1)
X_df=new_df = pd.DataFrame(new_df)
rf=RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_df, y)
feature_importance = rf.feature_importances_

# Tạo DataFrame để dễ dàng đọc kết quả
importance_df = pd.DataFrame({
    'Feature': X_df.columns,
    'Importance': feature_importance
})

# Sắp xếp và hiển thị kết quả
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df)


                      Feature  Importance
22       FunctionalAssessment    0.190598
25                        ADL    0.167770
21                       MMSE    0.121221
23           MemoryComplaints    0.087263
24         BehavioralProblems    0.051010
7                 DietQuality    0.031132
19             CholesterolHDL    0.030589
3                         BMI    0.030149
20   CholesterolTriglycerides    0.029844
8                SleepQuality    0.029625
5          AlcoholConsumption    0.028640
18             CholesterolLDL    0.028065
17           CholesterolTotal    0.027507
16                DiastolicBP    0.024625
0                         Age    0.024240
15                 SystolicBP    0.023796
6            PhysicalActivity    0.018183
2              EducationLevel    0.009888
1                      Gender    0.004372
14               Hypertension    0.004010
10      CardiovascularDisease    0.003865
4                     Smoking    0.003858
30              Forgetfulness    0

In [78]:
import joblib
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score

# Giả sử bạn đã có dữ liệu X và y
# X, y = ...

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Khởi tạo và huấn luyện mô hình XGBoost
xgb_model = XGBClassifier()
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'alpha': [0, 0.1, 0.5, 1],  
    'reg_lambda': [0, 1, 5, 10]  # Đảm bảo dùng 'reg_lambda' thay vì 'lambda'
}

# Khởi tạo GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Huấn luyện mô hình và tìm siêu tham số tối ưu
grid_search.fit(X_train, y_train)

# In kết quả
print(f'Best parameters: {grid_search.best_params_}')
print(f'Best score: {grid_search.best_score_}')

# Dự đoán trên tập huấn luyện và kiểm tra
y_train_pred = grid_search.best_estimator_.predict(X_train)
y_test_pred = grid_search.best_estimator_.predict(X_test)

# Tính độ chính xác
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f'Train Accuracy: {train_accuracy:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

# Cross-validation accuracy
cross_val_score_results = cross_val_score(grid_search.best_estimator_, X_train, y_train, cv=5, scoring='accuracy')
print(f'Cross-validation Accuracy: {cross_val_score_results.mean():.4f} +/- {cross_val_score_results.std():.4f}')

# Lưu mô hình tốt nhất
joblib.dump(grid_search.best_estimator_, 'alzheimers_model.pkl')
print("Mô hình đã được lưu vào tệp 'alzheimers_model.pkl'")

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


c:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best parameters: {'alpha': 0.5, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 50, 'reg_lambda': 1}
Best score: 0.9514551495016612
Train Accuracy: 0.9601
Test Accuracy: 0.9411
Cross-validation Accuracy: 0.9515 +/- 0.0150
Mô hình đã được lưu vào tệp 'alzheimers_model.pkl'


In [79]:
print(grid_search.best_estimator_.predict([[25,2.8,1,1,10]]))

[0]


In [58]:
results = permutation_importance(grid_search.best_estimator_, X_test, y_test, scoring='accuracy')

# Hiển thị kết quả
for i in range(len(results.importances_mean)):
    print(f"Feature {i}: {results.importances_mean[i]}")

Feature 0: 0.04651162790697676
Feature 1: 0.1048062015503876
Feature 2: 0.07410852713178295
Feature 3: 0.04961240310077519
Feature 4: 0.10666666666666665


In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

# Xây dựng mô hình Logistic Regression với L1 hoặc L2 regularization
logreg_model = LogisticRegression(solver='liblinear')  # 'liblinear' hỗ trợ L1 và L2

# Định nghĩa các tham số cho GridSearch
param_grid = {
    'penalty': ['l1', 'l2'],  # L1 hoặc L2 regularization
    'C': [0.1, 1, 10],  # Hệ số điều chỉnh độ mạnh của regularization
    'max_iter': [100, 200]  # Số lần lặp tối đa
}

# Khởi tạo GridSearchCV với Logistic Regression
grid_search = GridSearchCV(logreg_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Huấn luyện mô hình
grid_search.fit(X_train, y_train)

# Dự đoán trên dữ liệu huấn luyện và kiểm tra
y_train_pred = grid_search.best_estimator_.predict(X_train)
y_test_pred = grid_search.best_estimator_.predict(X_test)

# Tính toán độ chính xác
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f'Train Accuracy: {train_accuracy:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

# Đánh giá Cross-validation
cross_val_score_results = cross_val_score(grid_search.best_estimator_, X_train, y_train, cv=5, scoring='accuracy')
print(f'Cross-validation Accuracy: {cross_val_score_results.mean():.4f} +/- {cross_val_score_results.std():.4f}')

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Train Accuracy: 0.8557
Test Accuracy: 0.8264
Cross-validation Accuracy: 0.8530 +/- 0.0334


In [57]:
from sklearn.ensemble import RandomForestClassifier
rf_model=model = RandomForestClassifier()
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
}

# 3. Khởi tạo GridSearchCV với 5-fold cross-validation
grid_search_rf = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# 4. Tìm kiếm và huấn luyện mô hình
grid_search_rf.fit(X_train, y_train)

y_train_pred = grid_search_rf.best_estimator_.predict(X_train)
y_test_pred = grid_search_rf.best_estimator_.predict(X_test)

# Tính toán độ chính xác
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f'Train Accuracy: {train_accuracy:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

# Đánh giá Cross-validation
cross_val_score_results = cross_val_score(grid_search.best_estimator_, X_train, y_train, cv=5, scoring='accuracy')
print(f'Cross-validation Accuracy: {cross_val_score_results.mean():.4f} +/- {cross_val_score_results.std():.4f}')

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Train Accuracy: 0.9568
Test Accuracy: 0.9256
Cross-validation Accuracy: 0.8530 +/- 0.0334
